# Lab Assignment

In this lab assignment, you will improve your skills in scraping data from web pages, organizing the data in a desired format. To do so, we will use data that is provided by the Indeed company via their web pages.


You task will be to exectute the following blocks of code and try to understand what is happening after each line of the code. You will have to answer several questions to show your understanding.

Let us start with importing of the relevant packages.

Warning: I am using Python 3 for this lab assignment. If you are using Python 2, then some libraries may need to be changed., e.g., urllib -> urlib2.

In [ ]:
import urllib.request
import urllib.error
import bs4 #this is beautiful soup

import time
from pandas import Series
import pandas as pd
from pandas import DataFrame
import numpy as np
import re

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

As you see, you imported the package called `re`, which supports regular expression operations. You have already used the other packages. 

The following block will access and load the contents of an Indeed.com web page. It will probably give you a warning -- please ignore it.

In [ ]:
# Fixed url for job postings containing data scientist
url = 'https://www.indeed.fr/jobs?q=data+scientist&l=%C3%8Ele-de-France'
# read the website
source = urllib.request.urlopen(url).read()
# parse html code
bs_tree = bs4.BeautifulSoup(source)

** Question 1**:
 * What kind of page did you land on? Explain in a few sentences what is the content of this page.
 * What does the "?" and "&" in the url mean?
 * Explore the content of the web page. If you are using Firefox, open the page Inspector to understand how the underlying html code relates to the displayed content.
 * What pieces of the html code are responsible for showing the job title, company name, and job location?
 
The following block of code will figure out how many job ads are there for jobs that contain "data scientist' in the job title.

In [ ]:
# see how many job postings we found
job_count_string = bs_tree.find(id = 'searchCountPages').contents[0]
job_count_string = job_count_string.split()[-2]
print(("Search yielded %s hits." % (job_count_string)))

# not that job_count so far is still a string, 
# not an integer, and the , separator prevents 
# us from just casting it to int

job_count_digits = [int(d) for d in job_count_string if d.isdigit()]
job_count = np.sum([digit*(10**exponent) for digit, exponent in 
                    zip(job_count_digits[::-1], list(range(len(job_count_digits))))])

print(job_count)

** Question 2**:
* What does the "find(id = 'searchCountPages')" in the first line mean?
* What does the second line do?

**Question 3**: 
* Write a piece of code that finds the Salary Estimate object on the page


The following piece of code allows you to find the links to all the job ads on the page.

In [ ]:
# The website is only listing 10 results per page, 
# so we need to scrape them page after page
num_pages = int(np.ceil(job_count/10.0))

base_url = 'http://www.indeed.com'
job_links = []
for i in range(2): #do range(num_pages) if you want them all
    url = 'https://www.indeed.fr/emplois?q=data+scientist&l=%C3%8Ele-de-France&start=' + str(i*10)
    html_page = urllib.request.urlopen(url).read() 
    bs_tree = bs4.BeautifulSoup(html_page)
    job_link_area = bs_tree.find(id = 'resultsCol')

    job_postings = job_link_area.findAll("div")
    job_postings = [jp for jp in job_postings if str(jp.get('class')).find('row result')]
    
    job_ids = [jp.get('data-jk') for jp in job_postings]
    
    
    # go after each link
    for id in job_ids:
        if (id != None):
            job_links.append(base_url + '/rc/clk?jk=' + id)

    time.sleep(1)

print("We found a lot of jobs: ", len(job_links))

**Question 4**:
* explain why is `url` (first line in the for loop) defined as it is. What page are you landing at when `i = 3`?
* What does command `bs_tree.find(id = 'resultsCol')` do?
* What does command `''.join(jp.get('class'))` do?
* Explain the properties and content of `job_links`

**Question 5**:
* Reuse the code above to find links to the first 200 listed data science jobs

The following piece of code accesses and grabs the content of all ads in the `job_links` and extracts all the human-readable text tokens. Then, it counts on how many pages are words `mapreduce`, `spark`, and `visualization` used. It might take a few seconds to execute.

In [ ]:
skill_set = {'hadoop':0, 'spark':0, 'python':0}
counter = 0
for link in job_links:
    counter +=1  
    
    try:
        html_page = urllib.request.urlopen(link).read()
    except urllib.error.HTTPError:
        continue
    except urllib.error.URLError:
        continue
    except socket.error as error:
        print("Connection closed")
        continue

    html_text = re.sub("[^a-z.+3]"," ", html_page.decode('utf-8').lower()) # replace all but the listed characters
    
        
    for key in list(skill_set.keys()):
        if key in html_text:  
            skill_set[key] +=1
            
    if counter % 5 == 0:
        print(len(job_links) - counter)
        print(skill_set)
            
print(skill_set)

** Question 6**
* What does the `re.sub()` do in the above code? To answer this, you will have to go to the documentation of the regular expression package (find it on web).
* What is the meaning of `[^a-z.+3]`?
* What is the content of `skill_set`?

The following piece of code plots the counts of different skills in the processed ads.

In [ ]:
pseries = pd.Series(skill_set)
pseries.sort_values(ascending=False)

pseries.plot(kind = 'bar')
## set the title to Score Comparison
plt.title('Data Science Skills')
## set the x label
plt.xlabel('Skills')
## set the y label
plt.ylabel('Count')
## show the plot
plt.show()

**Question 7**:
- rerun the code to find the counts of the 3 data science skills on the first 200 job ads

## Data analysis of the housing market in the Paris area 

In the first part of the lab, you created `skill_set` dictionary that counts occurences of 3 important data science skills of the jobs in Ile de France. In the second part of this lab, your task will be to create a padas 'DataFrame' object containing information about the flats/houses that are available for rent in the paris area. For each offer, you will have to create columns such as: 
- Location
- Surface
- Number of rooms
- ...

To do so, we will be using seloger.com. Unlike the first part, where you were provided with snippets of codes that help you get familiarized with Web Scrapping skills, in this part you are left on your own to figure out how to do so. You can reduce your search to a specific area of paris, e.g., Paris 9, and for specific range of rent [700 euros,800 euros].

https://www.seloger.com/list.htm?projects=1&types=1,2&places=[{div:2238}]&price=700/800&enterprise=0&qsVersion=1.0

Note: I obtained the above URL by simply using the web page as a human via a browser, and specifying the criteria I needed to via the web form provided by seloger.com


### Deliverable: the modified .ipynb file that contains the answers to the questions